Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE       MCSE      ESS
alpha 154.6066575 0.270678217 0.0042797984 0.0041492373 1000
 beta   0.9049286 0.042638680 0.0006741767 0.0006531093 1000
sigma   5.1007848 0.195084007 0.0030845490 0.0034188340 1000

Quantiles:
          2.5%        25.0%       50.0%        75.0%       97.5%   
alpha 154.09700000 154.4220000 154.6035000 154.79200000 155.1370250
 beta   0.82291885   0.8754675   0.9051325   0.93456125   0.9877926
sigma   4.73280175   4.9662975   5.0959250   5.22602250   5.5059357



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.415 0.911598 4.87362; 155.239 0.920331 4.86811; … ; 154.756 0.951421 4.98863; 154.423 0.860477 5.0342]

Union{Missing, Float64}[154.545 0.893129 5.35471; 154.624 0.90257 5.00047; … ; 154.649 0.872991 4.95915; 154.74 0.905241 4.77537]

Union{Missing, Float64}[154.711 0.926219 4.94037; 154.894 0.84716 5.39336; … ; 154.816 0.884246 5.02044; 154.416 0.93465 5.06681]

Union{Missing, Float64}[154.697 0.886555 4.92582; 154.604 0.888125 5.038; … ; 154.39 0.886105 4.99736; 154.594 0.91524 5.06238]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE       MCSE      ESS
alpha 154.6066575 0.270678217 0.0042797984 0.0041492373 1000
 beta   0.9049286 0.042638680 0.0006741767 0.0006531093 1000
sigma   5.1007848 0.195084007 0.0030845490 0.0034188340 1000

Quantiles:
          2.5%        25.0%       50.0%        75.0%       97.5%   
alpha 154.09700000 154.4220000 154.6035000 154.79200000 155.1370250
 beta   0.82291885   0.8754675   0.9051325   0.93456125   0.9877926
sigma   4.73280175   4.9662975   5.0959250   5.22602250   5.5059357



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*